In [5]:
#automatically reload stuff
%load_ext autoreload
%autoreload 2

import simplejson
import Utils
import matplotlib.pyplot as plt
from SpatialPreprocessing import *
from Autoencoders import autoencode

from multiprocessing import Pool, cpu_count
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#read in the organ info and save it
spatial_files = load_spatial_files()
od = OrganData()
pdict = od.process_cohort_spatial_dict(spatial_files)
np_dict_to_json(pdict,Const.processed_organ_json)
print(pdict)
del pdict

missing doses set()
missing distances {10153, 205, 10159, 281, 285}
error reading patient 27
'<' not supported between instances of 'float' and 'str'
error reading patient 10029
bad operand type for unary -: 'str'
invalid patients [17, 39, 101, 115, 239, 10011, 10034, 10043, 10074, 10080, 10094, 10145, 10148, 10164, 10174, 10181, 2020]
{'organs': ['Bone_Hyoid', 'Brainstem', 'Cartlg_Thyroid', 'Cricoid', 'Cricopharyngeus', 'Esophagus_U', 'Glottic_Area', 'Hardpalate', 'Larynx', 'Lips_Lower', 'Lips_Upper', 'Mandible', 'Musc_Geniogloss', 'Musc_Masseter_L', 'Musc_Masseter_R', 'Musc_Sclmast_L', 'Musc_Sclmast_R', 'Oral_Cavity'], 'patients': {3: {'GTVp': {'volume': 13.1, 'mean_dose': 71.9, 'centroids': array([295.3126, 226.5868,  77.6118]), 'distances': array([15.25 , 39.12 ,  7.812, 17.75 , 20.86 , 36.53 , 32.22 , 58.88 ,
       13.67 , 77.06 , 82.1  , 13.35 ,    nan, 15.19 , 74.44 , -7.01 ,
       47.72 , 15.75 ], dtype=float16)}, 'GTVn': {'volume': 3.1, 'mean_dose': 71.27, 'centroids': array

In [7]:
#make sure the dict is being saved properly
pdata = load_pdict()
pdata 

{'organs': ['Bone_Hyoid',
  'Brainstem',
  'Cartlg_Thyroid',
  'Cricoid',
  'Cricopharyngeus',
  'Esophagus_U',
  'Glottic_Area',
  'Hardpalate',
  'Larynx',
  'Lips_Lower',
  'Lips_Upper',
  'Mandible',
  'Musc_Geniogloss',
  'Musc_Masseter_L',
  'Musc_Masseter_R',
  'Musc_Sclmast_L',
  'Musc_Sclmast_R',
  'Oral_Cavity'],
 'patients': {'3': {'GTVp': {'volume': 13.1,
    'mean_dose': 71.9,
    'centroids': [295.3126, 226.5868, 77.6118],
    'distances': [15.25,
     39.125,
     7.8125,
     17.75,
     20.859375,
     36.53125,
     32.21875,
     58.875,
     13.671875,
     77.0625,
     82.125,
     13.3515625,
     nan,
     15.1875,
     74.4375,
     -7.01171875,
     47.71875,
     15.75]},
   'GTVn': {'volume': 3.1,
    'mean_dose': 71.27,
    'centroids': [277.3846, 212.5056, 68.1895],
    'distances': [9.765625,
     31.890625,
     25.015625,
     42.5625,
     51.78125,
     71.0625,
     42.21875,
     29.84375,
     23.515625,
     62.5,
     60.28125,
     11.8828125,
 

In [8]:
#get a dict of denoised arrays for each type of organ
data = autoencode(pdata, ['distances','volume','centroids','mean_dose'], train=False)
print()
for k,v in data.items():
    print(k)
    for kk,vv in v.items():
        print(kk,'array->', 'mean:', np.round(vv.mean(),5),'std', np.round(vv.std(),5), 'shape:',vv.shape)
    print()

C:\Users\Andrew Wentzel\.conda\envs\qubbed_env\lib\site-packages\numpy\lib\shape_base.py:402: RuntimeWarning: All-NaN slice encountered
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))
C:\Users\Andrew Wentzel\.conda\envs\qubbed_env\lib\site-packages\numpy\lib\shape_base.py:379: RuntimeWarning: All-NaN slice encountered
  res = asanyarray(func1d(inarr_view[ind0], *args, **kwargs))


issue loading pretrained autoencoder ../resources/pytorch_models/autoencoder_distances_volume_centroids_mean_dose_221x19x23.pt training...
training stopped on epoch 2728
tensor(0.0463, grad_fn=<MeanBackward0>)

distances
denoised array-> mean: 40.14978 std 39.62381 shape: (221, 19, 18)
original array-> mean: 40.06758 std 39.81762 shape: (221, 19, 18)

volume
denoised array-> mean: 13.59766 std 27.92587 shape: (221, 19, 1)
original array-> mean: 13.3444 std 28.72972 shape: (221, 19, 1)

centroids
denoised array-> mean: 162.40158 std 87.55671 shape: (221, 19, 3)
original array-> mean: 162.51486 std 87.95145 shape: (221, 19, 3)

mean_dose
denoised array-> mean: 18.41216 std 21.10211 shape: (221, 19, 1)
original array-> mean: 17.55835 std 22.41069 shape: (221, 19, 1)



In [9]:
dists = data['distances']['denoised']
vols = data['volume']['denoised']
dists.shape

(221, 19, 18)

In [11]:
def local_tssim(x,y,v = None, w = None):
    #calculates local similarity within two numpy arrays
    #ignores structure of the windows
    #x, y are base variables (distances) for patients 1 and 2
    #v and w are volumes for patients 1 and 2
    #should all be 1-dimensional for original intended use
    c1 = .000001
    c2  = .000001
    x = x
    y = y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    covariance = np.cov(x,y)
    numerator = (2*mean_x*mean_y + c1) * (covariance[0,1] + covariance[1,0] + c2)
    denominator = (mean_x**2 + mean_y**2 + c1)*(np.var(x) + np.var(y) + c2)
    if v is not None and w is not None:
        mean_v = np.mean(v)
        mean_w = np.mean(w)
        numerator *= (2*mean_v*mean_w + c1)
        denominator *= (mean_v**2 + mean_w**2 + c1)
    if denominator > 0:
        return numerator/denominator
    else:
        print('error, zero denomiator in ssim function')
        return 0
    
def get_adjacency_list(dist_array, window_size = 50, gtvname = "GTV"):
    #takes distance array (n_patients) x (n_organs + gtv) x (n_organs)
    #specific dimensions are flexible but the above is the format I'm writting it for
    #returns a list of (n_organs + gtv) arrays with the indeces of organs withing window_size distance
    mean_dists = np.nanmean(dists,axis=0)
    adjacency_list = []
    for organ_row in mean_dists:
        #gets indeces for organs within a certian distance
        #will index like dist_array[:,organ_row_idx,adjacent_organs]
        adjacent_organs = np.argwhere(organ_row < window_size)
        adjacency_list.append(adjacent_organs.ravel())
    return adjacency_list

def pairwise_mean_local_similarity(d1, d2, v1, v2, adjacency_list, local_sim_func = None):
    #d1, d2 are min_distance arrays of shape (n_organs + <optional gtv>) x (n_organs)
    #v1 v2 are volume arrays of shape (n_organs + <optional_gtv>) x 1
    #adjacency list should be a list of (n_organs + <optional gtv>) indices for dim2
    #local sim func should take 2 vectors for distances and two volumes
    assert(d1.ndim == d2.ndim)
    sims = np.empty((len(adjacency_list),))
    if local_sim_func is None:
        local_sim_func = local_tssim
    for i,adjacency in enumerate(adjacency_list):
        od1 = d1[i,adjacency]
        od2 = d2[i,adjacency]
        ov1 = v1[i]
        ov2 = v2[i]
        sim = local_sim_func(od1,od2,ov1,ov2)
        sims[i] = sim
    return sims.mean()
        
def pairwise_tssim_job(p1, p2, dist_array, vol_array, adjacency_list):
    #wrapper for using similarity using multithreading
    #only works this way becuase it has to be pickled
    d1 = dist_array[p1]
    d2 = dist_array[p2]
    v1 = vol_array[p1]
    v2 = vol_array[p2]
    similarity = pairwise_mean_local_similarity(d1,d2,v1,v2,adjacency_list)
    return p1, p2, similarity

    
def get_similarity_matrix(dist_array, vol_array, sim_func = None, max_jobs = 8):
    #gets a n_items x n_items similarity matrix
    #pairwise sim should take idx 1, idx 2, distance array, volme array, adjacency list
    #default to tssim
    if sim_func is None:
        sim_func = pairwise_tssim_job
        
    x_items = dist_array.shape[0]
    adjacency_list = get_adjacency_list(dist_array)
    similarity_array = np.zeros((x_items, x_items))
    
    #get the number of available cpus
    #doesn't work on windows?  So i just assume all but one is free
    try:
        available_cpus = len(os.sched_getaffinity(0))
    except:
        available_cpus = cpu_count() - 2
    n_jobs = max(min(max_jobs, available_cpus), 1)
    with Pool(n_jobs) as pool:
        for p1 in range(x_items):
            score_results = []
            for p2 in range(p1 + 1,x_items):
                score_result = pool.apply_async(sim_func,
                                               args = (p1,p2,
                                                      dist_array,
                                                      vol_array,
                                                      adjacency_list)
                                               )
                score_results.append(score_result)
            for res in score_results:
                (p1,p2,sim) = res.get(10000)
                similarity_array[p1,p2] = sim
    similarity_array += similarity_array.transpose()
    return similarity_array
    
get_similarity_matrix(dists, vols)

TimeoutError: 

In [ ]:
vols[0]